In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
import sys
sys.path.insert(0,'/content/drive/MyDrive/RecSys/')
sys.path.append('/content/drive/MyDrive/RecSys/model/')

In [3]:
import math
import random as rnd
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.optim.lr_scheduler as lr_scheduler
from torch.autograd import Variable
from torch.utils.data import DataLoader, Subset,random_split,default_collate
from sklearn.model_selection import train_test_split
import time
import logging


In [4]:
import matplotlib.pyplot as plt
import seaborn as sns
from numba import jit

In [5]:
from Recommender import AEReco
from MovieLens_4Colab import Movie_100K, CombinedDataset



In [6]:
dataset = Movie_100K()

In [7]:
dataset[0]


{'user_id': 1,
 'age': 0.8,
 'gender': 0,
 'zip_code': array([ 0.00000000e+00,  5.00000000e-01,  8.66025404e-01,  1.00000000e+00,
         8.66025404e-01,  5.00000000e-01,  1.22464680e-16, -5.00000000e-01,
        -8.66025404e-01, -1.00000000e+00, -8.66025404e-01, -5.00000000e-01]),
 'occupation': array([ 0.        ,  0.27783279,  0.10412135, -0.238812  , -0.19361916,
         0.16625078,  0.25592374, -0.07034012, -0.28228459, -0.03544959,
         0.2689994 ,  0.13626052, -0.21793406, -0.21793406,  0.13626052,
         0.2689994 , -0.03544959, -0.28228459, -0.07034012,  0.25592374,
         0.16625078, -0.19361916, -0.238812  ,  0.10412135,  0.27783279]),
 'rating': 4,
 'timestamp': 878542420,
 'movie_id': 61,
 'date': array([1.2       , 0.5       , 0.8660254 , 0.5       , 0.97952994]),
 'genre': array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])}

In [8]:
model = AEReco()


 	 ----------- Model = Recommender with Autoencoder ------------

 	 	 Autoencoder = DAE 

 	 ----------- Model Loaded ------------
	 *Total Params* =  445235464
	 *Trainable Params* =  445235464


In [9]:
train_test_ratio = 0.8
np.random.seed(42)
indices = np.random.permutation(100000)
train_index = indices[:int(train_test_ratio*100000)]
test_index = indices[int(train_test_ratio*100000):]
train_dataset = Subset(dataset, train_index)
test_dataset = Subset(dataset, test_index)


In [10]:
#rating, user, item, mask = model.preprocessor(train_dataset, dataset)

In [11]:
#torch.save(mask, '/content/drive/MyDrive/RecSys/mask.pt')
#torch.save(rating, '/content/drive/MyDrive/RecSys/rating.pt')
#torch.save(user, '/content/drive/MyDrive/RecSys/user.pt')
#torch.save(item, '/content/drive/MyDrive/RecSys/item.pt')

In [12]:
rating = torch.load('/content/drive/MyDrive/RecSys/rating.pt')
user = torch.load('/content/drive/MyDrive/RecSys/user.pt')
item = torch.load('/content/drive/MyDrive/RecSys/item.pt')
mask = torch.load('/content/drive/MyDrive/RecSys/mask.pt')

<ipython-input-12-b930700b3dfa>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  rating = torch.load('/content/drive/MyDrive/RecSys/rating.pt')
<ipython-input-12-b930700b3df

The following way is when we attempt to see how the algorithm deal with the slow start issue.

In [13]:
#train_test_ratio = 0.8
#indices = np.random.permutation(user.shape[0])
#train_index = indices[:int(train_test_ratio*user.shape[0])]
#test_index = indices[int(train_test_ratio*user.shape[0]):]

In [14]:
#rating_train = rating[train_index]
#mask_train = mask[train_index]
#user_train = user[train_index]

#rating_test = rating[test_index]
#mask_test = mask[test_index]
#user_test = user[test_index]

In [15]:
print(f'item: {item.shape}')
print(f'user: {user.shape}')

item: torch.Size([1682, 26])
user: (943, 59)


In [16]:

def custom_collate(batch, batch_size, dataset):
    if len(batch) < batch_size:

        additional_samples_needed = batch_size - len(batch)
        indices = np.random.choice(len(dataset), additional_samples_needed)
        additional_data = [dataset[idx] for idx in indices]
        batch.extend(additional_data)
    return default_collate(batch)


In [17]:
combineset = CombinedDataset(user, rating, mask)
dataset = DataLoader(combineset, batch_size = 64, shuffle = True,
                     collate_fn=lambda x: custom_collate(x, 64, combineset))

In [ ]:
model.train(dataset, item, epoch = 90)
# with hidden_size = 256

Epoch 1: 100%|██████████| 15/15 [00:03<00:00,  3.96batch/s, loss=0.107]



	 Epoch : 1, RMSE = 1.9803
	 Training time for current epoch: 3.79 seconds


Epoch 2: 100%|██████████| 15/15 [00:03<00:00,  4.45batch/s, loss=0.114]



	 Epoch : 2, RMSE = 1.9261
	 Training time for current epoch: 3.38 seconds


Epoch 3: 100%|██████████| 15/15 [00:03<00:00,  4.29batch/s, loss=0.122]


...

Epoch 8: 100%|██████████| 15/15 [00:03<00:00,  4.47batch/s, loss=0.088]



	 Epoch : 8, RMSE = 1.7011
	 Training time for current epoch: 3.36 seconds


Epoch 9: 100%|██████████| 15/15 [00:03<00:00,  4.48batch/s, loss=0.112]



	 Epoch : 9, RMSE = 1.6862
	 Training time for current epoch: 3.36 seconds


Epoch 10:  93%|█████████▎| 14/15 [00:03<00:00,  4.46batch/s, loss=0.121]

In [ ]:
model.load_checkpoint(10)